In [ ]:
import astropy.units as u
from astropy.io import ascii
from collections import OrderedDict
import numpy as np
from numpy.polynomial.polynomial import polyval
import matplotlib.pyplot as plt
plt.style.use('apw-notebook')
%matplotlib inline
from scipy.optimize import minimize
from scipy.signal import argrelmin
from comoving_rv.longslit.wavelength import fit_spec_line
from comoving_rv.longslit.models import voigt_polynomial

import celerite
from celerite.modeling import Model
from celerite import terms, GP

In [ ]:
tbl = ascii.read("../data/mdm-spring-2017/processed/wavelength_guess.csv")#[1:]
# tbl.remove_row(np.argmin(np.abs(tbl['pixel']-506.84778)))
tbl

In [ ]:
tbl[-4]['pixel'] = 506.849811006

In [ ]:
class MeanModel(Model):

    def __init__(self, n_bg_coef, **p0):
        self._n_bg_coef = n_bg_coef
        self.parameter_names = (["a{}".format(i) for i in range(n_bg_coef)])
        super(MeanModel, self).__init__(**p0)

    def get_value(self, x):
        return polyval(x, np.atleast_1d([getattr(self, "a{}".format(i))
                                         for i in range(self._n_bg_coef)]))

In [ ]:
med_x = np.median(tbl['pixel'])
x = tbl['pixel'] - med_x
y = tbl['wavelength']

idx = np.argsort(x)
x = x[idx]
y = y[idx]

In [ ]:
# estimate background
a1 = (y[-1]-y[0])/(x[-1]-x[0]) # slope
a0 = y[-1] - a1*x[-1] # estimate constant term

In [ ]:
# initialize model
mean_model = MeanModel(n_bg_coef=2, a0=a0, a1=a1)
kernel = terms.Matern32Term(log_sigma=np.log(1.), log_rho=np.log(10.)) # MAGIC NUMBERs

# set up the gp model
gp = celerite.GP(kernel, mean=mean_model, fit_mean=True)
gp.compute(x, yerr=0.05) # need to do this
init_params = gp.get_parameter_vector()
print("Initial log-likelihood: {0}".format(gp.log_likelihood(y)))

In [ ]:
# Define a cost function
def neg_log_like(params, y, gp):
    gp.set_parameter_vector(params)
    ll = gp.log_likelihood(y)
    if np.isnan(ll):
        return np.inf
    return -ll

In [ ]:
# Fit for the maximum likelihood parameters
bounds = gp.get_parameter_bounds()
soln = minimize(neg_log_like, init_params, method="L-BFGS-B",
                bounds=bounds, args=(y, gp))
gp.set_parameter_vector(soln.x)
print("Success: {}, Final log-likelihood: {}".format(soln.success, -soln.fun))

In [ ]:
# Make the maximum likelihood prediction
x_grid = np.linspace(x.min(), x.max(), 1024)
mu, var = gp.predict(y, x_grid, return_var=True)
std = np.sqrt(var)

# Plot the maximum likelihood model
fig,ax = plt.subplots(1, 1, figsize=(8,8))

# data
ax.scatter(x + med_x, y, marker='o')

# mean model
ax.plot(x_grid+med_x, mean_model.get_value(x_grid), marker='', alpha=0.5)

# full GP model
gp_color = "#ff7f0e"
ax.plot(x_grid+med_x, mu, color=gp_color, marker='')
ax.fill_between(x_grid+med_x, mu+std, mu-std, color=gp_color,
                alpha=0.3, edgecolor="none")

ax.set_xlabel('pixel')
ax.set_ylabel(r'wavelength [$\AA$]')

fig.tight_layout()

In [ ]:
fit_par_dict = gp.get_parameter_dict()
fit_par_dict

In [ ]:
# pars = OrderedDict([('kernel:log_sigma', 1.35323184633239),
#                     ('kernel:log_rho', 10.9297580415762852),
#                     ('mean:a0', 6302.065931604905),
#                     ('mean:a1', -2.061369375358967)])
# gp.set_parameter_vector(list(pars.values()))

# Make the maximum likelihood prediction
# x_grid = np.linspace(x.min(), x.max(), 1024)
x_grid = np.linspace(0, 1600, 1024)- med_x
mu, var = gp.predict(y, x_grid, return_var=True)
std = np.sqrt(var)

_y_mean = mean_model.get_value(x)
_mu_mean = mean_model.get_value(x_grid)

# Plot the maximum likelihood model
fig,ax = plt.subplots(1, 1, figsize=(8,8))

# data
ax.scatter(x + med_x, y - _y_mean, marker='o')

# full GP model
gp_color = "#ff7f0e"
ax.plot(x_grid+med_x, mu - _mu_mean, color=gp_color, marker='')
ax.fill_between(x_grid+med_x, mu+std-_mu_mean, mu-std-_mu_mean, color=gp_color,
                alpha=0.3, edgecolor="none")

ax.set_xlabel('pixel')
ax.set_ylabel(r'wavelength [$\AA$]')

fig.tight_layout()

In [ ]:
# Make the maximum likelihood prediction
x_grid = np.linspace(0, 1600, 1024)- med_x
mu, var = gp.predict(y, x_grid, return_var=True)
std = np.sqrt(var)

y_mu, var = gp.predict(y, x, return_var=True)

# Plot the maximum likelihood model
fig,ax = plt.subplots(1, 1, figsize=(12,8))

# data
ax.scatter(x + med_x, y - y_mu, marker='o')

# full GP model
gp_color = "#ff7f0e"
ax.plot(x_grid+med_x, mu-mu, color=gp_color, marker='')
ax.fill_between(x_grid+med_x, std, -std, color=gp_color,
                alpha=0.3, edgecolor="none")

ax.set_xlabel('pixel')
ax.set_ylabel(r'wavelength residual [$\AA$]')

ax.set_ylim(-0.4, 0.4)
ax.axvline(683., zorder=-10, color='#666666', alpha=0.5)

ax2 = ax.twinx()
ax2.set_ylim([x/6563*300000 for x in ax.get_ylim()])
ax2.set_ylabel(r'velocity error at ${{\rm H}}_\alpha$ [{}]'.format((u.km/u.s).to_string(format='latex_inline')))

fig.tight_layout()

In [ ]:
mu,var = gp.predict(y, 815.-med_x, return_var=True)
mu, np.sqrt(var), np.sqrt(var) / mu * 300000

In [ ]:
mu,var = gp.predict(y, 687.63-med_x, return_var=True)
mu, np.sqrt(var), np.sqrt(var) / mu * 300000

In [ ]:
mu,var = gp.predict(y, 1172-med_x, return_var=True)
mu, np.sqrt(var), np.sqrt(var) / mu * 300000